In [ ]:
install.packages("DescTools")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proxy’, ‘rootSolve’, ‘e1071’, ‘lmom’, ‘mvtnorm’, ‘expm’, ‘Rcpp’, ‘Exact’, ‘gld’, ‘BH’




In [ ]:
library(dplyr)
library(tidyr)
library(tibble)
library(DescTools)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# **PART 3**

# Importing the Data

In [ ]:
scores <- data.frame(
  genre = rep(c("Pop", "Hip-Hop", "Rock", "Electronic"), each = 4),
  popularity_score = c(85, 88, 78, 80, 75, 72, 80, 77, 23, 11, 31, 8, 80, 82, 76, 75),
  danceability = c(7, 8, 6, 7, 6, 5, 7, 6, 5, 6, 4, 5, 8, 9, 7, 8),
  energy = c(8, 7, 7, 6, 9, 8, 7, 6, 8, 7, 6, 5, 9, 9, 8, 7)
)

scores

genre,popularity_score,danceability,energy
<chr>,<dbl>,<dbl>,<dbl>
Pop,85,7,8
Pop,88,8,7
Pop,78,6,7
Pop,80,7,6
Hip-Hop,75,6,9
Hip-Hop,72,5,8
Hip-Hop,80,7,7
Hip-Hop,77,6,6
Rock,23,5,8


# ANOVA test

In [ ]:
popularity_anova <- aov(popularity_score ~ genre, data = scores)
popularity_anova_sum = summary(popularity_anova)

popularity_anova_sum

            Df Sum Sq Mean Sq F value   Pr(>F)    
genre        3  11166    3722   94.58 1.29e-08 ***
Residuals   12    472      39                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [ ]:
danceability_anova <- aov(danceability ~ genre, data = scores)
danceability_anova_sum = summary(danceability_anova)

danceability_anova_sum

            Df Sum Sq Mean Sq F value  Pr(>F)   
genre        3     20   6.667      10 0.00139 **
Residuals   12      8   0.667                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [ ]:
energy_anova <- aov(energy ~ genre, data = scores)
energy_anova_sum = summary(energy_anova)

energy_anova_sum

            Df Sum Sq Mean Sq F value Pr(>F)
genre        3  6.687   2.229   1.814  0.198
Residuals   12 14.750   1.229               

# Calculating means and standard deviations for popularity_score and danceability
### (As the result related to these two attributes were significant.)

In [ ]:
summary_stats <- scores %>%
  group_by(genre) %>%
  summarize(mean_popularity = mean(popularity_score), sd_popularity = sd(popularity_score),
            mean_danceability = mean(danceability), sd_danceability = sd(danceability))

summary_stats

genre,mean_popularity,sd_popularity,mean_danceability,sd_danceability
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Electronic,78.25,3.304038,8,0.8164966
Hip-Hop,76.00,3.366502,6,0.8164966
Pop,82.75,4.573474,7,0.8164966
Rock,18.25,10.688779,5,0.8164966


# Determining the critical value for Tukey's HSD method
### (for significance-level=0.05, k=4 and df_e=12)

In [ ]:
alpha <- 0.05
df_e <- 12 # popularity_anova$df.residual
k <- 4 # length(unique(scores$genre))
q <- qtukey(1-alpha, df_e, k)

q

[1] 8.208283

# Calculating the HSD value using the formula: HSD = q * sqrt(MSE/n)

In [ ]:
# MSE
mse_popularity <- popularity_anova_sum[[1]]["Residuals","Mean Sq"]
mse_danceability <- danceability_anova_sum[[1]]["Residuals","Mean Sq"]


# Sample size (= 16)
n_popularity <- length(scores$popularity_score)
n_danceability <- length(scores$danceability)


# HSD
hsd_popularity <- q * sqrt(mse_popularity/n_popularity)
hsd_danceability <- q * sqrt(mse_danceability/n_danceability)

cat("hsd for popularity = ", hsd_popularity, "\n")
cat("hsd for danceability = ", hsd_danceability)

hsd for popularity =  12.87323 
hsd for danceability =  1.675509

# Calculating the difference between the means for each pair of groups

In [ ]:
genre_pairs <- crossing(genre1 = unique(scores$genre), genre2 = unique(scores$genre))

# Removing duplicates and pairs with the same genre
genre_pairs <- genre_pairs %>%
  filter(genre1 != genre2) %>%
  arrange(genre1)

# Filtering pairs with genre1 of one pair matching genre2 of another pair, and vice versa
genre_pairs <- genre_pairs %>%
  filter(!((genre_pairs$genre1 %in% genre_pairs$genre2 & genre_pairs$genre2 %in% genre_pairs$genre1) &
           (as.numeric(factor(genre_pairs$genre1)) > as.numeric(factor(genre_pairs$genre2))))) %>%
  distinct(genre1, genre2)

genre_pairs

genre1,genre2
<chr>,<chr>
Electronic,Hip-Hop
Electronic,Pop
Electronic,Rock
Hip-Hop,Pop
Hip-Hop,Rock
Pop,Rock


In [ ]:
for (i in 1:nrow(genre_pairs)) {
  genres <- genre_pairs[i, c("genre1", "genre2")]
  print(genres)

  mean_popularity <- summary_stats %>% filter(genre %in% genres) %>% pull(mean_popularity)
  print(mean_popularity)
  mean_danceability <- summary_stats %>% filter(genre %in% genres) %>% pull(mean_danceability)
  print(mean_danceability)

  diff_mean_popularity <- abs(mean_popularity[1] - mean_popularity[2])
  genre_pairs[i, "diff_mean_popularity"] <- diff_mean_popularity
  diff_mean_danceability <- abs(mean_danceability[1] - mean_danceability[2])
  genre_pairs[i, "diff_mean_danceability"] <- diff_mean_danceability
}

print(genre_pairs)

# A tibble: 1 × 2
  genre1     genre2 
  <chr>      <chr>  
1 Electronic Hip-Hop
[1] 78.25 76.00
[1] 8 6
# A tibble: 1 × 2
  genre1     genre2
  <chr>      <chr> 
1 Electronic Pop   
[1] 78.25 82.75
[1] 8 7
# A tibble: 1 × 2
  genre1     genre2
  <chr>      <chr> 
1 Electronic Rock  
[1] 78.25 18.25
[1] 8 5
# A tibble: 1 × 2
  genre1  genre2
  <chr>   <chr> 
1 Hip-Hop Pop   
[1] 76.00 82.75
[1] 6 7
# A tibble: 1 × 2
  genre1  genre2
  <chr>   <chr> 
1 Hip-Hop Rock  
[1] 76.00 18.25
[1] 6 5
# A tibble: 1 × 2
  genre1 genre2
  <chr>  <chr> 
1 Pop    Rock  
[1] 82.75 18.25
[1] 7 5
# A tibble: 6 × 4
  genre1     genre2  diff_mean_popularity diff_mean_danceability
  <chr>      <chr>                  <dbl>                  <dbl>
1 Electronic Hip-Hop                 2.25                      2
2 Electronic Pop                     4.5                       1
3 Electronic Rock                   60                         3
4 Hip-Hop    Pop                     6.75                      1
5 Hip-H

In [ ]:
genre_pairs

genre1,genre2,diff_mean_popularity,diff_mean_danceability
<chr>,<chr>,<dbl>,<dbl>
Electronic,Hip-Hop,2.25,2
Electronic,Pop,4.50,1
Electronic,Rock,60.00,3
Hip-Hop,Pop,6.75,1
Hip-Hop,Rock,57.75,1
Pop,Rock,64.50,2


# Dividing the absolute difference by the standard error of the difference -> t-value

In [ ]:
# Calculating the standard error of the difference
std_error_diff_popularity <- sqrt(2 * mse_popularity / n_popularity)
std_error_diff_danceability <- sqrt(2 * mse_danceability / n_danceability)

for (i in 1:nrow(genre_pairs)) {
  # Calculating the t-value for each pair
  t_value_popularity = abs(genre_pairs[i, "diff_mean_popularity"]) / std_error_diff_popularity
  genre_pairs[i, "t_value_popularity"] <- t_value_popularity
  t_value_danceability = abs(genre_pairs[i, "diff_mean_danceability"]) / std_error_diff_danceability
  genre_pairs[i, "t_value_danceability"] <- t_value_danceability
}

print(std_error_diff_popularity)
print(std_error_diff_danceability)
print(genre_pairs)

[1] 2.217943
[1] 0.2886751
# A tibble: 6 × 6
  genre1   genre2 diff_mean_popularity diff_mean_danceability t_value_popularity
  <chr>    <chr>                 <dbl>                  <dbl>              <dbl>
1 Electro… Hip-H…                 2.25                      2               1.01
2 Electro… Pop                    4.5                       1               2.03
3 Electro… Rock                  60                         3              27.1 
4 Hip-Hop  Pop                    6.75                      1               3.04
5 Hip-Hop  Rock                  57.8                       1              26.0 
6 Pop      Rock                  64.5                       2              29.1 
# ℹ 1 more variable: t_value_danceability <dbl>


In [ ]:
genre_pairs

genre1,genre2,diff_mean_popularity,diff_mean_danceability,t_value_popularity,t_value_danceability
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Electronic,Hip-Hop,2.25,2,1.014454,6.928203
Electronic,Pop,4.50,1,2.028907,3.464102
Electronic,Rock,60.00,3,27.052094,10.392305
Hip-Hop,Pop,6.75,1,3.043361,3.464102
Hip-Hop,Rock,57.75,1,26.037640,3.464102
Pop,Rock,64.50,2,29.081001,6.928203


# Comparing the HSD value and t-value
## (If the t-value is greater than the HSD value, then the difference between the means is considered statistically significant.)

In [ ]:
print(hsd_popularity)
print(hsd_danceability)

[1] 12.87323
[1] 1.675509


In [ ]:
genre_pairs$significant_popularity_diff_mean <- ifelse(abs(genre_pairs$t_value_popularity) > hsd_popularity, "Yes", "No")
genre_pairs$significant_danceability_diff_mean <- ifelse(abs(genre_pairs$t_value_danceability) > hsd_danceability, "Yes", "No")

genre_pairs

genre1,genre2,diff_mean_popularity,diff_mean_danceability,t_value_popularity,t_value_danceability,significant_popularity_diff_mean,significant_danceability_diff_mean
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
Electronic,Hip-Hop,2.25,2,1.014454,6.928203,No,Yes
Electronic,Pop,4.50,1,2.028907,3.464102,No,Yes
Electronic,Rock,60.00,3,27.052094,10.392305,Yes,Yes
Hip-Hop,Pop,6.75,1,3.043361,3.464102,No,Yes
Hip-Hop,Rock,57.75,1,26.037640,3.464102,Yes,Yes
Pop,Rock,64.50,2,29.081001,6.928203,Yes,Yes


In [ ]:
genre_pairs[, c('genre1', 'genre2', 'diff_mean_popularity', 'diff_mean_danceability', 'significant_popularity_diff_mean', 'significant_danceability_diff_mean')]

genre1,genre2,diff_mean_popularity,diff_mean_danceability,significant_popularity_diff_mean,significant_danceability_diff_mean
<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
Electronic,Hip-Hop,2.25,2,No,Yes
Electronic,Pop,4.50,1,No,Yes
Electronic,Rock,60.00,3,Yes,Yes
Hip-Hop,Pop,6.75,1,No,Yes
Hip-Hop,Rock,57.75,1,Yes,Yes
Pop,Rock,64.50,2,Yes,Yes


# **PART 4**

## 2-way ANOVA

In [ ]:
model <- lm(popularity_score ~ danceability * energy * genre, data = scores)
anova_2way = anova(model)

anova_2way

,Df,Sum Sq,Mean Sq,F value,Pr(>F)
,<int>,<dbl>,<dbl>,<dbl>,<dbl>
danceability,1,5089.50893,5089.508929,20358.03571,0.004461748
energy,1,244.22232,244.222321,976.88929,0.020361479
genre,3,5858.49471,1952.831571,7811.32628,0.008317154
danceability:energy,1,32.26765,32.267647,129.07059,0.055891869
danceability:genre,3,314.21270,104.737566,418.95026,0.035895334
energy:genre,3,80.51004,26.836680,107.34672,0.070803974
danceability:energy:genre,2,18.97115,9.485577,37.94231,0.114046058
Residuals,1,0.25000,0.250000,NA,NA


# **PART 5**

# Performing multiple linear regression analysis

In [ ]:
mlr <- lm(popularity_score ~ danceability + energy, data = scores)
mlr_sum = summary(mlr)

mlr_sum


Call:
lm(formula = popularity_score ~ danceability + energy, data = scores)

Residuals:
    Min      1Q  Median      3Q     Max 
-45.604 -10.759   5.406  13.196  24.131 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)  
(Intercept)   -42.033     37.692  -1.115   0.2850  
danceability   12.081      4.606   2.623   0.0211 *
energy          3.736      5.264   0.710   0.4905  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 22.02 on 13 degrees of freedom
Multiple R-squared:  0.4583,	Adjusted R-squared:  0.3749 
F-statistic: 5.499 on 2 and 13 DF,  p-value: 0.0186
